In [1]:
#%cd $HOME/projects/where_does_icl_happen
%cd /workspace/where_does_in-context-learning_happen_in_LLMs

/workspace/where_does_in-context-learning_happen_in_LLMs


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pip install tqdm > /dev/null 2>&1
!pip install omegaconf > /dev/null 2>&1
!pip install seaborn > /dev/null 2>&1
!pip install pynvml > /dev/null 2>&1
!pip install transformers > /dev/null 2>&1
!pip install sacrebleu > /dev/null 2>&1
!pip install nltk  > /dev/null 2>&1
!pip install accelerate > /dev/null 2>&1
!pip install -U bitsandbytes > /dev/null 2>&1

In [4]:
import os
from tqdm import tqdm
import seaborn as sns
import torch
import matplotlib.pyplot as plt
from src.utils import utils, build, io_utils
from src.datasets.collate_fn import CollateFn
from torch.utils.data import DataLoader
from omegaconf import OmegaConf
import seaborn as sns

In [5]:
os.environ['HF_HOME'] = "/workspace/.cache"

In [ ]:
args_model = OmegaConf.load('configs/model/masks/mask_context_from_TTF.yaml')
args_data = OmegaConf.load('configs/data/default.yaml')
args_format = OmegaConf.load('configs/format/instr_machine_translation.yaml')
args_model.model.model_size="gptn125M"
model, tokenizer = build.build_model_tok(args_model.model)

Loading model class: GPTNeoForCausalLMHack


In [ ]:
from src.datasets.prompt_dataset import PromptsDataset
promptbank, test_dataset = build.build_datasets_for_prompt(args_data.data)
prompt_ds = PromptsDataset(args_format.format, 
                           promptbank,
                           test_dataset,
                           seed=0, 
                           nprompts=1,
                           ntest=1)
collate_fn = CollateFn(tokenizer)
dataloader = DataLoader(prompt_ds, collate_fn=collate_fn, batch_size=1)

In [17]:
utils.reset_mask(model)
args_model.model.mask_layer = 0

In [18]:
from src.utils.utils import build_causal_mask_per_batch

In [ ]:
with torch.no_grad():
    for j, batch in enumerate(tqdm(dataloader)):        
        build_causal_mask_per_batch(args_model.model, model, batch)    
        outputs_w_mask = model.generate(batch['input_ids'],
                                 attention_mask=batch['attention_mask'],
                                 pad_token_id=tokenizer.pad_token_id,
                                 output_attentions=True,
                                 return_dict_in_generate=True,
                                 output_scores=True)
        break

In [ ]:
out_attn = outputs_w_mask['attentions'][0][0][0][0]
sns.heatmap(out_attn.cpu().numpy(), cmap="binary")

Illustration of redundancy in self-attention computation based on our masking
setup 